# PID TikZ Pipeline (Local GPU-Friendly Notebook)
This notebook mirrors the end-to-end pipeline in a laptop-friendly, restartable format. It keeps everything self-contained by downloading models into the working directory and shielding the run from GPU out-of-memory events.

## 1. Load Configuration and Working Paths
Define reusable parameters, ensure required packages are present, and prepare the local directory structure where artifacts and models will be stored.

In [1]:
# Section 1: imports, package checks, and configuration
import json
import logging
import os
import subprocess
import sys
from pathlib import Path

print(f"Using Python {sys.version}")

REQUIRED_PACKAGES = [
    "torch",
    "transformers>=4.44.0",
    "accelerate>=0.33.0",
    "bitsandbytes>=0.43.1",
    "huggingface_hub>=0.23.0",
]

def ensure_package(spec: str) -> None:
    package_key = spec.split("==")[0].split(">=")[0].replace("-", "_")
    try:
        __import__(package_key)
    except ImportError:
        print(f"Installing {spec} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", spec])

for requirement in REQUIRED_PACKAGES:
    ensure_package(requirement)

from datetime import datetime  # noqa: E402 (import after potential installs)

PROJECT_ROOT = Path.cwd()
ARTIFACTS_DIR = PROJECT_ROOT / "notebook_artifacts"
MODELS_DIR = PROJECT_ROOT / "local_models"
TIKZ_INPUT_DIR = PROJECT_ROOT / "tikz_inputs"
for directory in (ARTIFACTS_DIR, MODELS_DIR, TIKZ_INPUT_DIR):
    directory.mkdir(parents=True, exist_ok=True)

CONFIG = {
    "tikz_path": "D:\\P&ID LTTS\\test\\main.tex",
    "tikz_to_text_model": "meta-llama/Llama-3.1-8B-Instruct",
    "text_to_tikz_model": "nllg/detikzify-v2.5-8b",
    "hf_token": os.environ.get("HF_TOKEN"),
    "compile_pdf": False,
    "job_name": f"pid_notebook_run_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}",
    "max_tokens_description": 1024,
    "temperature_description": 0.3,
    "top_p_description": 0.9,
    "repetition_penalty_description": 1.05,
    "max_tokens_regen": 768,
    "temperature_regen": 0.2,
    "top_p_regen": 0.95,
    "repetition_penalty_regen": 1.0,
}

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)8s | %(message)s")

RUN_STATE: dict = {}
MODEL_CACHE: dict = {}

print(f"Project root        : {PROJECT_ROOT}")
print(f"Artifacts directory : {ARTIFACTS_DIR}")
print(f"Models directory    : {MODELS_DIR}")
print(f"TikZ source path    : {CONFIG['tikz_path']}")

Using Python 3.9.21 (main, Dec 11 2024, 16:35:24) [MSC v.1929 64 bit (AMD64)]


c:\Users\lenovo\anaconda3\envs\ai-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Project root        : d:\P&ID LTTS
Artifacts directory : d:\P&ID LTTS\notebook_artifacts
Models directory    : d:\P&ID LTTS\local_models
TikZ source path    : D:\P&ID LTTS\test\main.tex


## 2. Ensure GPU Availability with Memory-Safe Fallback
Inspect CUDA availability, report memory capacity, and register helpers that deallocate GPU memory so the notebook can continue after an out-of-memory exception.

In [2]:
# Section 2: device inspection and OOM helpers
import torch

DEVICE_INFO = {
    "device": torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"),
    "memory_gb": None,
}

if torch.cuda.is_available():
    props = torch.cuda.get_device_properties(0)
    DEVICE_INFO["memory_gb"] = round(props.total_memory / (1024 ** 3), 2)
    print(f"CUDA device detected: {props.name} with {DEVICE_INFO['memory_gb']} GB")
else:
    print("CUDA device not detected; computations will fall back to CPU.")

def recover_from_oom(stage: str, error: BaseException) -> None:
    """Release GPU resources and log the out-of-memory event."""
    logging.warning("[OOM] %s encountered: %s", stage, error)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    print(f"Recovered from OOM in {stage}; continuing on CPU where necessary.")

CUDA device detected: Quadro M1000M with 4.0 GB


In [3]:
# Section 3: model download utilities
from typing import Optional
import re
import time
import json
import socket

from huggingface_hub import snapshot_download, hf_hub_download

try:  # Newer huggingface_hub versions expose HfHubHTTPError at top level
    from huggingface_hub import HfHubHTTPError  # type: ignore[attr-defined]
except Exception:
    try:  # Older releases keep it under huggingface_hub.utils
        from huggingface_hub.utils import HfHubHTTPError  # type: ignore
    except Exception:  # Fallback when the helper is absent altogether
        class HfHubHTTPError(Exception):
            """Fallback exception used when huggingface_hub lacks HfHubHTTPError."""

try:
    from huggingface_hub.errors import LocalEntryNotFoundError  # type: ignore[attr-defined]
except Exception:
    try:  # Some releases expose it under huggingface_hub.utils
        from huggingface_hub.utils import LocalEntryNotFoundError  # type: ignore
    except Exception:
        class LocalEntryNotFoundError(Exception):
            """Fallback raised when huggingface_hub omits LocalEntryNotFoundError."""

try:
    from huggingface_hub.utils import RepositoryNotFoundError  # type: ignore
except Exception:  # pragma: no cover - older releases reuse HfHubHTTPError
    RepositoryNotFoundError = HfHubHTTPError  # type: ignore

try:
    from requests.exceptions import ConnectionError as RequestsConnectionError, ConnectTimeout
except Exception:  # pragma: no cover - requests should be present, but guard anyway
    RequestsConnectionError = ConnectTimeout = type("RequestsConnectionError", (Exception,), {})


def _safe_repo_dirname(repo_id: str) -> str:
    # Make a filesystem-safe name for the repo identifier
    return re.sub(r"[^0-9A-Za-z._-]", "_", repo_id)


def _clean_incomplete_files(target_dir: Path, cache_key: str) -> None:
    """Remove any partially downloaded blobs so retries start fresh."""
    partials = list(target_dir.rglob("*.incomplete"))
    if not partials:
        return
    for partial in partials:
        try:
            partial.unlink()
        except OSError as unlink_err:
            logging.warning("[%s] Could not remove partial file %s: %s", cache_key, partial, unlink_err)


def _download_missing_shards(
    repo_id: str,
    cache_key: str,
    target_dir: Path,
    token: Optional[str],
    *,
    max_retry: int,
) -> None:
    """Ensure sharded safetensors referenced by the index are present."""
    index_path = target_dir / "model.safetensors.index.json"
    if not index_path.exists():
        return

    try:
        index_data = json.loads(index_path.read_text(encoding="utf-8"))
    except Exception as err:
        logging.warning("[%s] Unable to parse safetensor index for shard reconciliation: %s", cache_key, err)
        return

    weight_map = index_data.get("weight_map", {})
    if not weight_map:
        return

    required_files = {Path(filename).name for filename in weight_map.values()}
    missing_files = [name for name in required_files if not (target_dir / name).exists()]
    if not missing_files:
        return

    logging.info("[%s] Detected %s missing shard(s); downloading individually.", cache_key, len(missing_files))
    for filename in missing_files:
        for attempt in range(1, max_retry + 1):
            try:
                hf_hub_download(
                    repo_id=repo_id,
                    filename=filename,
                    repo_type="model",
                    local_dir=target_dir.as_posix(),
                    token=token,
                )
                break
            except HfHubHTTPError as err:
                status_code = getattr(getattr(err, "response", None), "status_code", None)
                if status_code == 401:
                    raise RuntimeError(
                        f"Unauthorized when fetching shard '{filename}' of '{repo_id}'. Provide a valid HF_TOKEN with access to the repository."
                    ) from err
                raise
            except LocalEntryNotFoundError as err:
                logging.warning(
                    "[%s] Shard %s attempt %s/%s failed due to transient hub lookup error; retrying soon...",
                    cache_key,
                    filename,
                    attempt,
                    max_retry,
                )
                if attempt == max_retry:
                    raise
                time.sleep(min(30, 5 * attempt))
            except (RequestsConnectionError, ConnectTimeout, socket.gaierror) as err:
                logging.warning(
                    "[%s] Shard %s attempt %s/%s failed due to network resolution error (%s); retrying soon...",
                    cache_key,
                    filename,
                    attempt,
                    max_retry,
                    err,
                )
                if attempt == max_retry:
                    raise RuntimeError(
                        f"Network failure while downloading shard '{filename}' from '{repo_id}'. Please verify connectivity and retry."
                    ) from err
                time.sleep(min(30, 5 * attempt))
            except RuntimeError as err:
                if "CAS service error" in str(err):
                    logging.warning(
                        "[%s] Shard %s attempt %s/%s hit CAS service error; retrying soon...",
                        cache_key,
                        filename,
                        attempt,
                        max_retry,
                    )
                    if attempt == max_retry:
                        raise
                    time.sleep(min(30, 5 * attempt))
                else:
                    raise
        else:  # pragma: no cover - defensive safeguard
            raise RuntimeError(f"Failed to download shard '{filename}' after {max_retry} retries.")


def ensure_model_local(repo_id: str, cache_key: str, token: Optional[str] = None, *, max_retry: int = 3) -> Path:
    target_dir = MODELS_DIR / _safe_repo_dirname(repo_id)
    marker_file = target_dir / ".completed"
    if marker_file.exists() and target_dir.exists():
        print(f"[{cache_key}] Using cached weights at {target_dir}")
        MODEL_CACHE[cache_key] = target_dir
        return target_dir

    print(f"[{cache_key}] Downloading snapshot {repo_id} into {target_dir}")
    target_dir.mkdir(parents=True, exist_ok=True)

    for attempt in range(1, max_retry + 1):
        _clean_incomplete_files(target_dir, cache_key)
        try:
            snapshot_download(
                repo_id=repo_id,
                repo_type="model",
                local_dir=target_dir.as_posix(),
                token=token,
            )
            _download_missing_shards(
                repo_id,
                cache_key,
                target_dir,
                token,
                max_retry=max_retry,
            )
            marker_file.touch()
            MODEL_CACHE[cache_key] = target_dir
            print(f"[{cache_key}] Download complete into {target_dir}")
            return target_dir
        except RepositoryNotFoundError as err:
            raise RuntimeError(
                f"Model '{repo_id}' not found or access denied. Accept the repository license and ensure HF_TOKEN is set if required."
            ) from err
        except HfHubHTTPError as err:
            if getattr(err, "response", None) is not None and getattr(err.response, "status_code", None) == 401:
                raise RuntimeError(
                    f"Unauthorized when fetching '{repo_id}'. Provide a valid HF_TOKEN with access to the repository."
                ) from err
            raise
        except LocalEntryNotFoundError as err:
            logging.warning(
                "[%s] Snapshot attempt %s/%s failed due to transient hub lookup error; retrying soon...",
                cache_key,
                attempt,
                max_retry,
            )
            if attempt == max_retry:
                logging.error("[%s] Exhausted retries due to repeated hub lookup failures.", cache_key)
                raise
            time.sleep(min(30, 5 * attempt))
        except (RequestsConnectionError, ConnectTimeout, socket.gaierror) as err:
            logging.warning(
                "[%s] Snapshot attempt %s/%s failed due to network resolution error (%s); retrying soon...",
                cache_key,
                attempt,
                max_retry,
                err,
            )
            if attempt == max_retry:
                logging.error("[%s] Exhausted retries due to persistent network resolution errors.", cache_key)
                raise RuntimeError(
                    f"Network failure while downloading '{repo_id}'. Please verify internet connectivity and retry."
                ) from err
            time.sleep(min(30, 5 * attempt))
        except RuntimeError as err:
            if "CAS service error" in str(err):
                logging.warning(
                    "[%s] Snapshot attempt %s/%s failed due to CAS service error; retrying soon...",
                    cache_key,
                    attempt,
                    max_retry,
                )
                if attempt == max_retry:
                    logging.error("[%s] Exhausted retries due to persistent CAS service error.", cache_key)
                    raise
                time.sleep(min(30, 5 * attempt))
            else:
                raise


MODEL_CACHE.clear()

tikz_repo = CONFIG["tikz_to_text_model"]
text2tikz_repo = CONFIG["text_to_tikz_model"]

try:
    tikz_path = ensure_model_local(tikz_repo, "tikz_to_text", CONFIG.get("hf_token"))
except Exception as e:
    logging.exception("Failed to fetch tikz->text model: %s", e)
    raise

try:
    text2tikz_path = ensure_model_local(text2tikz_repo, "text_to_tikz", CONFIG.get("hf_token"))
except Exception as e:
    logging.exception("Failed to fetch text->tikz model: %s", e)
    raise

print("Model cache entries:")
for key, path in MODEL_CACHE.items():
    print(f"  {key}: {path}")

[tikz_to_text] Using cached weights at d:\P&ID LTTS\local_models\meta-llama_Llama-3.1-8B-Instruct
[text_to_tikz] Using cached weights at d:\P&ID LTTS\local_models\nllg_detikzify-v2.5-8b
Model cache entries:
  tikz_to_text: d:\P&ID LTTS\local_models\meta-llama_Llama-3.1-8B-Instruct
  text_to_tikz: d:\P&ID LTTS\local_models\nllg_detikzify-v2.5-8b
[text_to_tikz] Using cached weights at d:\P&ID LTTS\local_models\nllg_detikzify-v2.5-8b
Model cache entries:
  tikz_to_text: d:\P&ID LTTS\local_models\meta-llama_Llama-3.1-8B-Instruct
  text_to_tikz: d:\P&ID LTTS\local_models\nllg_detikzify-v2.5-8b


## 4. Initialise TikZ-to-Text Model with Graceful OOM Handling
Load the analysis model from the local cache, preferring 4-bit quantisation when available and falling back to CPU if GPU memory becomes constrained.

In [ ]:
# Section 4: safe model loader for TikZ -> text
from dataclasses import dataclass
from typing import Any, Dict, Optional

from transformers import AutoModelForCausalLM, AutoTokenizer

try:
    from transformers import BitsAndBytesConfig
    BNB_AVAILABLE = True
except ImportError:  # bitsandbytes not present on CPU-only machines
    BNB_AVAILABLE = False


@dataclass
class GenerationDefaults:
    max_new_tokens: int
    temperature: float
    top_p: float
    repetition_penalty: float


class SafeCausalLM:
    def __init__(self, label: str, local_dir: Path, generation_defaults: GenerationDefaults) -> None:
        self.label = label
        self.local_dir = Path(local_dir)
        self.generation_defaults = generation_defaults
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(self.local_dir, use_fast=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self._load_model(force_cpu=False)

    def _quant_config(self) -> Any:
        if self.device.type != "cuda" or not BNB_AVAILABLE:
            return None
        return BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        )

    def _load_model(self, force_cpu: bool) -> None:
        target_device = torch.device("cpu") if force_cpu or self.device.type != "cuda" else torch.device("cuda")
        quant_config = None if target_device.type == "cpu" else self._quant_config()
        torch_dtype = torch.bfloat16 if target_device.type == "cuda" else torch.float32
        try:
            self.model = AutoModelForCausalLM.from_pretrained(
                self.local_dir,
                device_map="auto" if target_device.type == "cuda" else None,
                torch_dtype=torch_dtype,
                quantization_config=quant_config,
            )
            self.device = target_device
            print(f"[{self.label}] Loaded on {self.device} with 4-bit={quant_config is not None}")
        except (torch.cuda.OutOfMemoryError, RuntimeError) as err:
            if target_device.type == "cuda" and "out of memory" in str(err).lower():
                recover_from_oom(f"{self.label} loading", err)
                self._load_model(force_cpu=True)
            else:
                raise
        except ValueError as err:
            err_str = str(err)
            if target_device.type == "cuda" and (
                "llm_int8_enable_fp32_cpu_offload" in err_str
                or "dispatched on the cpu" in err_str.lower()
            ):
                logging.warning(
                    "[%s] Falling back to CPU load because quantised weights could not stay on GPU: %s",
                    self.label,
                    err,
                )
                self._load_model(force_cpu=True)
            else:
                raise

    def generate(self, prompt: str, overrides: Optional[Dict[str, Any]] = None) -> str:
        params = {
            "max_new_tokens": self.generation_defaults.max_new_tokens,
            "temperature": self.generation_defaults.temperature,
            "top_p": self.generation_defaults.top_p,
            "repetition_penalty": self.generation_defaults.repetition_penalty,
        }
        if overrides:
            params.update(overrides)

        attempt_cpu_fallback = False
        while True:
            try:
                inputs = self.tokenizer(prompt, return_tensors="pt")
                input_length = inputs["input_ids"].shape[-1]
                if self.device.type == "cuda":
                    inputs = {k: v.to(self.device) for k, v in inputs.items()}
                with torch.inference_mode():
                    output = self.model.generate(
                        **inputs,
                        do_sample=params["temperature"] > 0,
                        temperature=params["temperature"],
                        top_p=params["top_p"],
                        max_new_tokens=params["max_new_tokens"],
                        repetition_penalty=params["repetition_penalty"],
                        eos_token_id=self.tokenizer.eos_token_id,
                        pad_token_id=self.tokenizer.pad_token_id,
                    )
                generated = output[0, input_length:]
                return self.tokenizer.decode(generated, skip_special_tokens=True).strip()
            except (torch.cuda.OutOfMemoryError, RuntimeError) as err:
                if "out of memory" in str(err).lower() and self.device.type == "cuda":
                    recover_from_oom(f"{self.label} inference", err)
                    self.model.to("cpu")
                    self.device = torch.device("cpu")
                    attempt_cpu_fallback = True
                    continue
                raise
            finally:
                if attempt_cpu_fallback:
                    attempt_cpu_fallback = False


tikz_to_text_runner = SafeCausalLM(
    label="TikZ->Text",
    local_dir=MODEL_CACHE["tikz_to_text"],
    generation_defaults=GenerationDefaults(
        max_new_tokens=CONFIG["max_tokens_description"],
        temperature=CONFIG["temperature_description"],
        top_p=CONFIG["top_p_description"],
        repetition_penalty=CONFIG["repetition_penalty_description"],
    ),
)

2025-10-21 11:31:38,712 |     INFO | NumExpr defaulting to 8 threads.
2025-10-21 11:31:53,700 |     INFO | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
2025-10-21 11:31:53,734 |  WARNING | [TikZ->Text] Falling back to CPU load because quantised weights could not stay on GPU: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 
2025-10-21 11:31:53,700 |     INFO | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid O

## 5. Generate Natural-Language Description from TikZ Chunks
Split the TikZ source into manageable slices, construct prompts, and run the TikZ-to-text model while absorbing transient GPU memory errors.

In [ ]:
# Section 5: chunk TikZ input and produce narrative description
import textwrap


def chunk_text(text: str, max_chars: int = 6000, overlap: int = 500) -> list[str]:
    if len(text) <= max_chars:
        return [text]
    slices: list[str] = []
    stride = max(max_chars - overlap, 1)
    for start in range(0, len(text), stride):
        slices.append(text[start : start + max_chars])
    return slices


def build_description_prompt(tikz_snippet: str, ordinal: int) -> str:
    return textwrap.dedent(
        f"""
        <s>[INST]\nYou are a senior plant instrumentation engineer. Provide meticulous natural-language instructions to recreate a process and instrumentation diagram.\n\nTikZ snippet #{ordinal}:\n```tikz\n{tikz_snippet}\n```\n\nInclude:\n1. Numbered reconstruction steps covering all equipment, piping, instrumentation, and safety elements.\n2. Connectivity, flow direction, and signal semantics for each tag.\n3. Relative layout cues (e.g., left/right/up/down) to aid sketching.\n4. A concise summary paragraph at the end.\n[/INST]
        """
    ).strip()


tikz_path = Path(CONFIG["tikz_path"])
if not tikz_path.exists():
    sample_tikz = textwrap.dedent(
        r"""
        \begin{tikzpicture}[>=stealth]
          \draw[thick] (0,0) rectangle (2,2);
          \draw[thick,->] (2,1) -- (3.5,1) node[right]{Process flow};
          \node[draw,circle,minimum size=0.8cm] at (1,1) {P101};
          \node[draw,diamond,minimum size=0.8cm] at (0.5,1.8) {FIC-01};
          \draw[dashed] (0.5,1.8) -- (1,1.4);
        \end{tikzpicture}
        """
    ).strip()
    tikz_path.parent.mkdir(parents=True, exist_ok=True)
    tikz_path.write_text(sample_tikz, encoding="utf-8")
    print(f"No TikZ input found. Created a sample diagram at {tikz_path} for demonstration.")

CONFIG["tikz_path"] = tikz_path

tikz_source = tikz_path.read_text(encoding="utf-8")
RUN_STATE["tikz_path"] = str(tikz_path)
RUN_STATE["tikz_source"] = tikz_source

snippets = chunk_text(tikz_source)
print(f"Total TikZ chunks: {len(snippets)}")

description_parts: list[str] = []
for idx, snippet in enumerate(snippets, start=1):
    prompt = build_description_prompt(snippet, idx)
    try:
        response = tikz_to_text_runner.generate(prompt)
        description_parts.append(textwrap.dedent(response).strip())
        print(f"Chunk {idx} processed.")
    except Exception as err:  # let unexpected issues bubble up after logging
        logging.exception("Description generation failed on chunk %s: %s", idx, err)
        raise

final_description = "\n\n".join(part for part in description_parts if part)
RUN_STATE["description"] = final_description
print(f"Description length: {len(final_description.split())} words")

## 6. Initialise Text-to-TikZ Model with Graceful OOM Handling
Load the AutomaTikZ generator from the local cache using the same defensive strategy so the notebook can recover from GPU constraints.

In [ ]:
# Section 6: load AutomaTikZ generator
text_to_tikz_runner = SafeCausalLM(
    label="Text->TikZ",
    local_dir=MODEL_CACHE["text_to_tikz"],
    generation_defaults=GenerationDefaults(
        max_new_tokens=CONFIG["max_tokens_regen"],
        temperature=CONFIG["temperature_regen"],
        top_p=CONFIG["top_p_regen"],
        repetition_penalty=CONFIG["repetition_penalty_regen"],
    ),
)
print("Text->TikZ model ready on", text_to_tikz_runner.device)

## 7. Regenerate TikZ Code from Description
Feed the generated instructions into AutomaTikZ, buffering partial output so retries can continue even if GPU memory pressure forces the model onto CPU.

In [ ]:
# Section 7: convert description back into TikZ
if not RUN_STATE.get("description"):
    raise ValueError("Description text missing. Rerun Section 5 before regenerating TikZ.")

try:
    regenerated_tikz = text_to_tikz_runner.generate(
        RUN_STATE["description"],
        overrides={
            "max_new_tokens": CONFIG["max_tokens_regen"],
            "temperature": CONFIG["temperature_regen"],
            "top_p": CONFIG["top_p_regen"],
            "repetition_penalty": CONFIG["repetition_penalty_regen"],
        },
    )
    RUN_STATE["regenerated_tikz"] = regenerated_tikz
    print(f"Regenerated TikZ length: {len(regenerated_tikz.split())} tokens")
except Exception as err:
    logging.exception("Text-to-TikZ generation failed: %s", err)
    raise

## 8. Optionally Compile TikZ to PDF with Robust Logging
Invoke `pdflatex` only when requested, capturing output and skipping gracefully if TeX tooling is unavailable on the machine.

In [ ]:
# Section 8: optional PDF compilation
pdf_output_path = None
if CONFIG["compile_pdf"]:
    if "regenerated_tikz" not in RUN_STATE:
        raise ValueError("No regenerated TikZ found. Run Section 7 before compiling.")
    tex_document = """\\documentclass[tikz,border=5pt]{standalone}
\\usepackage{tikz}
\\usepackage{pgfplots}
\\pgfplotsset{compat=1.18}
\\begin{document}
%s
\\end{document}
""" % RUN_STATE["regenerated_tikz"]
    tex_path = ARTIFACTS_DIR / f"{CONFIG['job_name']}.tex"
    tex_path.write_text(tex_document, encoding="utf-8")
    try:
        result = subprocess.run(
            [
                "pdflatex",
                "-interaction=nonstopmode",
                "-halt-on-error",
                f"-output-directory={ARTIFACTS_DIR.as_posix()}",
                tex_path.as_posix(),
            ],
            check=True,
            capture_output=True,
            text=True,
        )
        print(result.stdout)
        pdf_output_path = ARTIFACTS_DIR / f"{CONFIG['job_name']}.pdf"
        if pdf_output_path.exists():
            RUN_STATE["compiled_pdf"] = str(pdf_output_path)
            print(f"PDF generated at {pdf_output_path}")
    except FileNotFoundError:
        logging.warning("pdflatex not installed. Skipping PDF compilation.")
    except subprocess.CalledProcessError as err:
        logging.error("pdflatex failed: %s", err.stdout)
else:
    print("PDF compilation skipped (set CONFIG['compile_pdf'] = True to enable).")

## 9. Persist Outputs and Summarise Artifacts
Write intermediate files (TikZ copy, description, regenerated TikZ, optional PDF) and a JSON manifest so downstream scripts can locate artifacts easily.

In [ ]:
# Section 9: persist artefacts and emit summary
summary = {
    "tikz_source_path": str(CONFIG["tikz_path"]),
    "artifacts_dir": str(ARTIFACTS_DIR),
    "tikz_to_text_model": CONFIG["tikz_to_text_model"],
    "text_to_tikz_model": CONFIG["text_to_tikz_model"],
    "description_tokens": len(RUN_STATE.get("description", "").split()),
}

extracted_path = ARTIFACTS_DIR / "extracted_tikz.tex"
extracted_path.write_text(RUN_STATE.get("tikz_source", ""), encoding="utf-8")
summary["extracted_tikz"] = str(extracted_path)

description_path = ARTIFACTS_DIR / "generated_description.txt"
description_path.write_text(RUN_STATE.get("description", ""), encoding="utf-8")
summary["description_file"] = str(description_path)

if "regenerated_tikz" in RUN_STATE:
    regen_path = ARTIFACTS_DIR / "regenerated_tikz.tex"
    regen_path.write_text(RUN_STATE["regenerated_tikz"], encoding="utf-8")
    summary["regenerated_tikz_file"] = str(regen_path)

if "compiled_pdf" in RUN_STATE:
    summary["compiled_pdf"] = RUN_STATE["compiled_pdf"]

summary_path = ARTIFACTS_DIR / "pipeline_summary.json"
summary_path.write_text(json.dumps(summary, indent=2), encoding="utf-8")
print("Summary written to", summary_path)
print(json.dumps(summary, indent=2))